In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Conv2DTranspose, Dropout, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import MeanIoU

In [2]:
# Task 1: Dataset Acquisition and Preprocessing
def load_and_preprocess_data():
    # Load the Oxford-IIIT Pet dataset
    (train_ds_raw, val_ds_raw, test_ds_raw), info = tfds.load(
        'oxford_iiit_pet',
        split=['train[:80%]', 'train[80%:]', 'test'],
        with_info=True,
        shuffle_files=True,
        as_supervised=True  # Returns (image, mask) tuples
    )

    def preprocess(image, mask):
        # Resize images and masks to 128x128
        image = tf.image.resize(image, (128, 128))
        mask = tf.image.resize(mask, (128, 128), method='nearest')
        # Normalize images to [0, 1]
        image = tf.cast(image, tf.float32) / 255.0
        # Adjust mask labels (subtract 1 to make them start from 0)
        mask -= 1
        mask = tf.cast(mask, tf.int32)
        return image, mask

    # Batch and prefetch datasets
    batch_size = 32
    train_ds = train_ds_raw.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    val_ds = val_ds_raw.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    test_ds = test_ds_raw.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return train_ds, val_ds, test_ds, info

In [3]:
# Load and preprocess the dataset
train_ds, val_ds, test_ds, info = load_and_preprocess_data()

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:   0%|          | 0/2 [00:00<?, ? url/s]
Extraction completed...: 0 file [03:28, ? file/s]
Dl Size...: 0 MiB [03:28, ? MiB/s]
Dl Completed...:   0%|          | 0/2 [03:28<?, ? url/s]


SSLError: HTTPSConnectionPool(host='thor.robots.ox.ac.uk', port=443): Max retries exceeded with url: /~vgg/data/pets/annotations.tar.gz (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:992)')))

In [ ]:
# Task 2: Build a CNN for Semantic Segmentation
def build_model(input_shape=(128, 128, 3), num_classes=3):
    inputs = Input(shape=input_shape)

    # Encoder (Downsampling Path)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)  # Max Pooling
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)  # Max Pooling
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)  # Max Pooling

    # Bottleneck
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)

    # Decoder (Upsampling Path)
    x = Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)

    # Output Layer
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

# Build the model
model = build_model()
model.summary()

In [ ]:
# Task 5: Train the Model
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=SparseCategoricalCrossentropy(),
              metrics=['accuracy', MeanIoU(num_classes=3)])

# Train the model
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=20,
                    batch_size=32)

In [ ]:
# Task 6: Evaluate the Model
# Evaluate on the test set
test_loss, test_accuracy, test_iou = model.evaluate(test_ds)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test IoU: {test_iou}")

In [ ]:
# Task 7: Visualize Results
# Plot training and validation loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation IoU
plt.subplot(1, 2, 2)
plt.plot(history.history['mean_io_u'], label='Training IoU')
plt.plot(history.history['val_mean_io_u'], label='Validation IoU')
plt.title('IoU Curve')
plt.xlabel('Epoch')
plt.ylabel('IoU')
plt.legend()
plt.show()

# Visualize segmentation results on test images
def visualize_predictions(dataset, model, num_samples=5):
    for images, masks in dataset.take(num_samples):
        predictions = model.predict(images)
        predicted_masks = tf.argmax(predictions, axis=-1)

        plt.figure(figsize=(15, 5))
        for i in range(num_samples):
            plt.subplot(1, num_samples, i + 1)
            plt.imshow(images[i])
            plt.title('Input Image')
            plt.axis('off')

            plt.subplot(2, num_samples, i + 1 + num_samples)
            plt.imshow(predicted_masks[i], cmap='viridis')
            plt.title('Predicted Mask')
            plt.axis('off')
        plt.show()

# Visualize predictions on the test set
visualize_predictions(test_ds, model)